<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/chatbot_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carichiamo il corpus di testo

In [0]:
import urllib.request, json 

with urllib.request.urlopen("https://raw.githubusercontent.com/ProfAI/nlp00/master/8%20-%20Chatbot/data/corpus.json") as url:
    data = url.read().decode()


In [83]:
corpus = json.loads(data)
corpus["intents"][0]

{'patterns': ['Ciao', 'Salve', 'hei', 'Ci sei ?', 'Buongiorno', 'Buonasera'],
 'responses': ['Ciao amico', 'Hei, felice di vederti', 'Eccoti qui !'],
 'tag': 'saluti'}

### Preprocessiamo i dati

In [84]:
import spacy
!python -m spacy download it_core_news_sm


    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



In [85]:
nlp = spacy.load("it_core_news_sm")

dictionary = set({})
tags = []

docs = []


for intent in corpus["intents"]:
  
  for pattern in intent["patterns"]:
    
    pattern = pattern.lower()
    tokens = nlp(pattern)
    doc = ""
    
    for token in tokens:
      if(not token.is_punct and not token.is_stop):
        doc+=" "+token.lemma_
        dictionary.add(token.lemma_)
        
    if(len(doc)>0):
      docs.append(doc.rstrip()) 
      tags.append(intent["tag"])
  
print("Lunghezza del dizionario: %d" % len(dictionary))
print(docs)
print(tags)

Lunghezza del dizionario: 30
[' ciao', ' salvo', ' hei', ' buongiorno', ' buonasera', ' addio', ' arrivederci', ' buono giornata', ' prestare', ' vedere', ' ringraziare', ' fantasticare', " consigliare un'allenamento addominale", ' volere allenare addominale', ' dammi esercizio addominale', ' volere allenarmi', ' fammi allenare', ' consigliami esercizio', ' chiamare', ' qual essere nome', ' presentare', ' sentire', ' buttare']
['saluti', 'saluti', 'saluti', 'saluti', 'saluti', 'arrivederci', 'arrivederci', 'arrivederci', 'arrivederci', 'arrivederci', 'grazie', 'grazie', 'workout-addominali', 'workout-addominali', 'workout-addominali', 'workout', 'workout', 'workout', 'presentazione', 'presentazione', 'presentazione', 'come-stai', 'come-stai']


### Bag of Words

In [86]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
X = bow.fit_transform(docs)
X.shape

(23, 31)

In [87]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(tags)
y[:5]

array([4, 4, 4, 4, 4])

In [88]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(y.reshape(-1, 1))
y.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(23, 7)

In [0]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation="relu", input_dim=X.shape[1]))
model.add(Dense(8, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [92]:
model.fit(X,y, epochs=500, batch_size=8)

Epoch 1/500
23/23 [==============================] - 0s 13ms/step - loss: 1.9407 - acc: 0.1739
Epoch 2/500
23/23 [==============================] - 0s 525us/step - loss: 1.9359 - acc: 0.1304
Epoch 3/500
23/23 [==============================] - 0s 1ms/step - loss: 1.9312 - acc: 0.1304
Epoch 4/500
23/23 [==============================] - 0s 579us/step - loss: 1.9274 - acc: 0.1304
Epoch 5/500
23/23 [==============================] - 0s 398us/step - loss: 1.9239 - acc: 0.1304
Epoch 6/500
23/23 [==============================] - 0s 401us/step - loss: 1.9200 - acc: 0.1304
Epoch 7/500
23/23 [==============================] - 0s 520us/step - loss: 1.9158 - acc: 0.1739
Epoch 8/500
23/23 [==============================] - 0s 641us/step - loss: 1.9123 - acc: 0.1739
Epoch 9/500
23/23 [==============================] - 0s 581us/step - loss: 1.9088 - acc: 0.1739
Epoch 10/500
23/23 [==============================] - 0s 550us/step - loss: 1.9053 - acc: 0.1739
Epoch 11/500
23/23 [======================

### Testiamo il chatbot

In [0]:
def preprocess(sentence):
  
  tokens = nlp(sentence)
  doc = ""
  
  for token in tokens:
    if(not token.is_punct and not token.is_stop):
      doc+=" "+token.lemma_
  
  x = bow.transform([doc])
  
  return x

In [0]:
def chatbot(sentence):
  
  x = preprocess(sentence)
  y = model.predict_classes(x)
  tag = le.inverse_transform(y)
  
  return get_response(tag)

In [0]:
from random import choice

def get_response(tag):
  
  for intent in corpus["intents"]:
    if(intent["tag"]==tag):
      return choice(intent["responses"])

In [98]:
sentence = None

while(sentence!="bye" and sentence!=""):
  sentence = input("Tu: ")
  response = chatbot(sentence)
  print("Chatbot: "+response)

Tu: ciao !
Chatbot: Ciao amico
Tu: come stai oggi ?
Chatbot: Hei, felice di vederti
Tu: come ti senti oggi amico ?
Chatbot: Mi sento in splendida forma
Tu: Fantastico !
Chatbot: Ciao amico
Tu: voglio allenare gli addominali
Chatbot: Ecco qualche esercizio per gli addominali: 10 crunches, 10 situp, 10 reverse crunch. Ripeti per 5 volte
Tu: consigliamo un'allenamento
Chatbot: Ecco qualche esercizio per gli addominali: 10 crunches, 10 situp, 10 reverse crunch. Ripeti per 5 volte
Tu: bye
Chatbot: Hei, felice di vederti


### input multipli

In [0]:
def preprocess(sentence):
  
  tokens = nlp(sentence)
  docs = []
  
  for sent in tokens.sents:
    
    doc = ""
    
    for token in sent:
      if(not token.is_punct and not token.is_stop):
        doc+=" "+token.lemma_
    
    docs.append(doc)
    
  X = bow.transform(docs)
  
  return X

In [0]:
from random import choice

def get_response(tags):
  
  response = ""
  
  for tag in tags:
  
    for intent in corpus["intents"]:
      if(intent["tag"]==tag):
        response+=choice(intent["responses"])+" "
    
  return response

In [104]:
sentence = None

while(sentence!="bye" and sentence!=""):
  sentence = input("Tu: ")
  response = chatbot(sentence)
  print("Chatbot: "+response)

Tu: ciao
Chatbot: Ciao amico 
Tu: Ciao ! Come ti senti oggi ?
Chatbot: Eccoti qui ! Mi sento in splendida forma 
Tu: Buongiorno, vorrei allenarmi
Chatbot: Ecco il tuo allenamento di oggi: 60 secondi di jumping jack, 10 piegamenti sulle braccia, 20 squats. Ripeti per 5 volte. 
Tu: Buongiorno. Vorrei allenarmi.
Chatbot: Eccoti qui ! A più tardi 
Tu: Ciao. Vorrei allenarmi
Chatbot: Ciao amico A presto 
Tu: dammi un'allenamento
Chatbot: Ecco qualche esercizio per gli addominali: 10 crunches, 10 situp, 10 reverse crunch. Ripeti per 5 volte 
Tu: bye
Chatbot: Ciao amico 
